#PROYECTO CAPSTONE SEMANA 5


In [4]:
#SE IMPORTAN LIBERIAS
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
from geopy.geocoders import Nominatim
import urllib.request
import json
from bs4 import BeautifulSoup
from urllib.request import urlopen
import requests
from pandas.io.json import json_normalize
import matplotlib.cm as cm
import matplotlib.colors as colors
import matplotlib.cm as cm
import matplotlib.pyplot as plt
import matplotlib.colors as colors
%matplotlib inline
from sklearn.cluster import KMeans
import folium

#PARA ESTE PROYECTO VAMOS A TRABAJAR CON NUEVA YOKR, ASÍ QUYE LO VAMOS A IMPORTAR DESDE AQUÍ 
https://maps.princeton.edu/catalog/nyu-2451-34572

In [5]:
with open('/content/nyu-2451-34572-geojson.json') as json_data:
    newyork_data = json.load(json_data)

In [6]:
neighborhoods_data = newyork_data['features']
#Se establencen las columnas del dataframe
column_names = ['Borough', 'Neighborhood', 'Latitude', 'Longitude'] 

#Se guardan en un nuevo dataframe que llamamos neighborhoods
neighborhoods = pd.DataFrame(columns=column_names)

for data in neighborhoods_data:
    borough = neighborhood_name = data['properties']['borough'] 
    neighborhood_name = data['properties']['name']
        
    neighborhood_latlon = data['geometry']['coordinates']
    neighborhood_lat = neighborhood_latlon[1]
    neighborhood_lon = neighborhood_latlon[0]
    
    neighborhoods = neighborhoods.append({'Borough': borough,
                                          'Neighborhood': neighborhood_name,
                                          'Latitude': neighborhood_lat,
                                          'Longitude': neighborhood_lon}, ignore_index=True)

<ipython-input-6-3c5095482864>:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  neighborhoods = neighborhoods.append({'Borough': borough,
<ipython-input-6-3c5095482864>:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  neighborhoods = neighborhoods.append({'Borough': borough,
<ipython-input-6-3c5095482864>:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  neighborhoods = neighborhoods.append({'Borough': borough,
<ipython-input-6-3c5095482864>:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  neighborhoods = neighborhoods.append({'Borough': borough,
<ipython-input-6-3c5095482864>:16: FutureWarning: The frame.append method is deprecated 

In [7]:
#Mostramos el nuevo dataframe creado
neighborhoods.head()

,Borough,Neighborhood,Latitude,Longitude
0,Bronx,Wakefield,40.894705,-73.847201
1,Bronx,Co-op City,40.874294,-73.829939
2,Bronx,Eastchester,40.887556,-73.827806
3,Bronx,Fieldston,40.895437,-73.905643
4,Bronx,Riverdale,40.890834,-73.912585


In [8]:
address = 'New York City, NY'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of New York City are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of New York City are 40.7127281, -74.0060152.


In [9]:
#Creamos un mapa de Nueva York y lo mostramos
manhattan_data = neighborhoods[neighborhoods['Borough'] == 'Manhattan'].reset_index(drop=True)
manhattan_data.head()

,Borough,Neighborhood,Latitude,Longitude
0,Manhattan,Marble Hill,40.876551,-73.910660
1,Manhattan,Chinatown,40.715618,-73.994279
2,Manhattan,Washington Heights,40.851903,-73.936900
3,Manhattan,Inwood,40.867684,-73.921210
4,Manhattan,Hamilton Heights,40.823604,-73.949688


In [10]:
import folium
#Creamos un mapa de N.Y. empleando valores de latitud y longitud
map_newyork = folium.Map(location=[latitude, longitude], zoom_start=10)

# Añadimos puntos del mapa
for lat, lng, borough, neighborhood in zip(manhattan_data['Latitude'], manhattan_data['Longitude'], manhattan_data['Borough'], manhattan_data['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_newyork)  
    
map_newyork

#USANDO FOURSQUARE

In [11]:
import urllib
def getNearbyVenues(names, latitudes, longitudes, radius=5000, categoryIds=''):
    try:
        venues_list=[]
        for name, lat, lng in zip(names, latitudes, longitudes):
            #print(name)

            # create the API request URL
            url = 'https://api.foursquare.com/v2/venues/search?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, VERSION, lat, lng, radius, LIMIT)

            if (categoryIds != ''):
                url = url + '&categoryId={}'
                url = url.format(categoryIds)

            # make the GET request
            response = requests.get(url).json()
            results = response["response"]['venues']

            # return only relevant information for each nearby venue
            for v in results:
                success = False
                try:
                    category = v['categories'][0]['name']
                    success = True
                except:
                    pass

                if success:
                    venues_list.append([(
                        name, 
                        lat, 
                        lng, 
                        v['name'], 
                        v['location']['lat'], 
                        v['location']['lng'],
                        v['categories'][0]['name']
                    )])

        nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
        nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude',  
                  'Venue Category']
    
    except:
        print(url)
        print(response)
        print(results)
        print(nearby_venues)

    return(nearby_venues)

In [33]:
LIMIT = 500 
radius = 5000 
CLIENT_ID = 'ZMHWBS0SR12Z3YDYVHJVTZPRK3U1ZP3I2TYQAJ5CU3JUHMB5'
CLIENT_SECRET = 'H3TT0XT3P5TIAFCV1Y2UUVLF42N44DICNKLUELK34H2TKLFR'
VERSION = '20181020'

In [34]:
#Sushi = 4bf58dd8d48988d1d2941735
neighborhoods = neighborhoods[neighborhoods['Borough'] == 'Manhattan'].reset_index(drop=True)
newyork_venues_sushi = getNearbyVenues(names=neighborhoods['Neighborhood'], latitudes=neighborhoods['Latitude'], longitudes=neighborhoods['Longitude'], radius=1000, categoryIds='4bf58dd8d48988d1d2941735')
newyork_venues_sushi.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Marble Hill,40.876551,-73.910660,Planet Tokyo,40.886233,-73.909479,Sushi Restaurant
1,Chinatown,40.715618,-73.994279,Time,40.715675,-73.994039,Sushi Restaurant
2,Chinatown,40.715618,-73.994279,zutto Japanese American Bar,40.720485,-73.996423,Japanese Restaurant
3,Chinatown,40.715618,-73.994279,Sushi Hatsune,40.715994,-73.992859,Sushi Restaurant
4,Chinatown,40.715618,-73.994279,O Ramen & Dim Sum M Yuki Sushi,40.716055,-73.992305,Sushi Restaurant


In [14]:
newyork_venues_sushi.shape

(1217, 7)

In [15]:
def addToMap(df, color, existingMap):
    for lat, lng, local, venue, venueCat in zip(df['Venue Latitude'], df['Venue Longitude'], df['Neighborhood'], df['Venue'], df['Venue Category']):
        label = '{} ({}) - {}'.format(venue, venueCat, local)
        label = folium.Popup(label, parse_html=True)
        folium.CircleMarker(
            [lat, lng],
            radius=5,
            popup=label,
            color=color,
            fill=True,
            fill_color=color,
            fill_opacity=0.7).add_to(existingMap)

In [16]:
#Imprimimos el mapa 
map_newyork_sushi = folium.Map(location=[latitude, longitude], zoom_start=10)
addToMap(newyork_venues_sushi, 'red', map_newyork_sushi)

map_newyork_sushi

In [17]:
def addColumn(startDf, columnTitle, dataDf):
    grouped = dataDf.groupby('Neighborhood').count()
    
    for n in startDf['Neighborhood']:
        try:
            startDf.loc[startDf['Neighborhood'] == n,columnTitle] = grouped.loc[n, 'Venue']
        except:
            startDf.loc[startDf['Neighborhood'] == n,columnTitle] = 0

In [18]:
manhattan_grouped = newyork_venues_sushi.groupby('Neighborhood').count()
manhattan_grouped
#print('There are {} uniques categories.'.format(len(newyork_venues_sushi['Venue Category'].unique())))

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Battery Park City,25,25,25,25,25,25
Carnegie Hill,24,24,24,24,24,24
Central Harlem,3,3,3,3,3,3
Chelsea,47,47,47,47,47,47
Chinatown,35,35,35,35,35,35
Civic Center,40,40,40,40,40,40
Clinton,43,43,43,43,43,43
East Harlem,3,3,3,3,3,3
East Village,50,50,50,50,50,50


#EN ESTE PASO VAMOS A TRABAJAR  CON CADA VECINDARIO

In [19]:
# Usamos one hot encodging
manhattan_onehot = pd.get_dummies(newyork_venues_sushi[['Venue Category']], prefix="", prefix_sep="")

# Agregamos una nueva columna  llamada Neighborhood
manhattan_onehot['Neighborhood'] = newyork_venues_sushi['Neighborhood'] 

#Ahora la movemos para que se ubique en  la primera posición 
fixed_columns = [manhattan_onehot.columns[-1]] + list(manhattan_onehot.columns[:-1])
manhattan_onehot = manhattan_onehot[fixed_columns]

#Imprimimos 
manhattan_onehot.head()

,Neighborhood,American Restaurant,Asian Restaurant,Bakery,Chinese Restaurant,Cocktail Bar,Deli / Bodega,Fish Market,Grocery Store,Hawaiian Restaurant,Indian Chinese Restaurant,Japanese Restaurant,Karaoke Bar,Korean Restaurant,Noodle House,Ramen Restaurant,Restaurant,Sake Bar,Seafood Restaurant,Smoothie Shop,Steakhouse,Sushi Restaurant,Vegetarian / Vegan Restaurant
0,Marble Hill,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
1,Chinatown,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
2,Chinatown,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
3,Chinatown,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
4,Chinatown,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0


In [20]:
manhattan_grouped = manhattan_onehot.groupby('Neighborhood').mean().reset_index()
manhattan_grouped


,Neighborhood,American Restaurant,Asian Restaurant,Bakery,Chinese Restaurant,Cocktail Bar,Deli / Bodega,Fish Market,Grocery Store,Hawaiian Restaurant,Indian Chinese Restaurant,Japanese Restaurant,Karaoke Bar,Korean Restaurant,Noodle House,Ramen Restaurant,Restaurant,Sake Bar,Seafood Restaurant,Smoothie Shop,Steakhouse,Sushi Restaurant,Vegetarian / Vegan Restaurant
0,Battery Park City,0.00,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.080000,0.000000,0.000000,0.040000,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.880000,0.000000
1,Carnegie Hill,0.00,0.041667,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.041667,0.125000,0.000000,0.000000,0.000000,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.750000,0.041667
2,Central Harlem,0.00,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000
3,Chelsea,0.00,0.042553,0.00,0.000000,0.000000,0.000000,0.021277,0.000000,0.000000,0.000000,0.085106,0.000000,0.000000,0.000000,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.829787,0.021277
4,Chinatown,0.00,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.114286,0.000000,0.000000,0.000000,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.885714,0.000000
5,Civic Center,0.00,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.075000,0.000000,0.000000,0.025000,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.900000,0.000000
6,Clinton,0.00,0.023256,0.00,0.023256,0.023256,0.000000,0.000000,0.000000,0.000000,0.000000,0.069767,0.000000,0.023256,0.000000,0.00,0.00,0.000000,0.023256,0.000000,0.000000,0.813953,0.000000
7,East Harlem,0.00,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000
8,East Village,0.00,0.000000,0.00,0.000000,0.000000,0.000000,0.020000,0.000000,0.000000,0.000000,0.060000,0.000000,0.000000,0.000000,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.900000,0.020000
9,Financial District,0.00,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.153846,0.000000,0.000000,0.000000,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.846154,0.000000


In [21]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [22]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = manhattan_grouped['Neighborhood']

for ind in np.arange(manhattan_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(manhattan_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Battery Park City,Sushi Restaurant,Japanese Restaurant,Noodle House,American Restaurant,Korean Restaurant,Steakhouse,Smoothie Shop,Seafood Restaurant,Sake Bar,Restaurant
1,Carnegie Hill,Sushi Restaurant,Japanese Restaurant,Vegetarian / Vegan Restaurant,Indian Chinese Restaurant,Asian Restaurant,Noodle House,Steakhouse,Smoothie Shop,Seafood Restaurant,Sake Bar
2,Central Harlem,Sushi Restaurant,American Restaurant,Asian Restaurant,Steakhouse,Smoothie Shop,Seafood Restaurant,Sake Bar,Restaurant,Ramen Restaurant,Noodle House
3,Chelsea,Sushi Restaurant,Japanese Restaurant,Asian Restaurant,Vegetarian / Vegan Restaurant,Fish Market,Noodle House,Steakhouse,Smoothie Shop,Seafood Restaurant,Sake Bar
4,Chinatown,Sushi Restaurant,Japanese Restaurant,American Restaurant,Korean Restaurant,Steakhouse,Smoothie Shop,Seafood Restaurant,Sake Bar,Restaurant,Ramen Restaurant


#CLUSTERING

In [23]:
# Empezamos usando 5
kclusters = 5

manhattan_grouped_clustering = manhattan_grouped.drop('Neighborhood', 1)

# Usamos k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(manhattan_grouped_clustering)

# Revisamos las etiquetas generadas por cada fila
kmeans.labels_[0:10]

<ipython-input-23-b1258ed834fe>:4: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  manhattan_grouped_clustering = manhattan_grouped.drop('Neighborhood', 1)
/usr/local/lib/python3.9/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


array([0, 4, 2, 1, 0, 0, 1, 2, 0, 4], dtype=int32)

In [24]:
# Añanadimos las etiquetas
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

manhattan_merged = manhattan_data
manhattan_merged = manhattan_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

manhattan_merged.head()

,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Manhattan,Marble Hill,40.876551,-73.910660,2,Sushi Restaurant,American Restaurant,Asian Restaurant,Steakhouse,Smoothie Shop,Seafood Restaurant,Sake Bar,Restaurant,Ramen Restaurant,Noodle House
1,Manhattan,Chinatown,40.715618,-73.994279,0,Sushi Restaurant,Japanese Restaurant,American Restaurant,Korean Restaurant,Steakhouse,Smoothie Shop,Seafood Restaurant,Sake Bar,Restaurant,Ramen Restaurant
2,Manhattan,Washington Heights,40.851903,-73.936900,3,Sushi Restaurant,Japanese Restaurant,American Restaurant,Korean Restaurant,Steakhouse,Smoothie Shop,Seafood Restaurant,Sake Bar,Restaurant,Ramen Restaurant
3,Manhattan,Inwood,40.867684,-73.921210,2,Sushi Restaurant,American Restaurant,Asian Restaurant,Steakhouse,Smoothie Shop,Seafood Restaurant,Sake Bar,Restaurant,Ramen Restaurant,Noodle House
4,Manhattan,Hamilton Heights,40.823604,-73.949688,2,Sushi Restaurant,American Restaurant,Asian Restaurant,Steakhouse,Smoothie Shop,Seafood Restaurant,Sake Bar,Restaurant,Ramen Restaurant,Noodle House


In [25]:
# Creamos un mapa
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

#Establecemos un esquema de color para cada clúster
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# Añadimos marcas
markers_colors = []
for lat, lon, poi, cluster in zip(manhattan_merged['Latitude'], manhattan_merged['Longitude'], manhattan_merged['Neighborhood'], manhattan_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

#AHORA IMPRIMIMOS  LOS DATAFRAMES PARA CADA CLUSTER LABEL , EMPEZANDO DESDE EL 0 HASTA EL 4

In [26]:
manhattan_merged.loc[manhattan_merged['Cluster Labels'] == 0, manhattan_merged.columns[[1] + list(range(5, manhattan_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,Chinatown,Sushi Restaurant,Japanese Restaurant,American Restaurant,Korean Restaurant,Steakhouse,Smoothie Shop,Seafood Restaurant,Sake Bar,Restaurant,Ramen Restaurant
18,Greenwich Village,Sushi Restaurant,Japanese Restaurant,Sake Bar,American Restaurant,Korean Restaurant,Steakhouse,Smoothie Shop,Seafood Restaurant,Restaurant,Ramen Restaurant
19,East Village,Sushi Restaurant,Japanese Restaurant,Vegetarian / Vegan Restaurant,Fish Market,Korean Restaurant,Steakhouse,Smoothie Shop,Seafood Restaurant,Sake Bar,Restaurant
20,Lower East Side,Sushi Restaurant,Japanese Restaurant,American Restaurant,Korean Restaurant,Steakhouse,Smoothie Shop,Seafood Restaurant,Sake Bar,Restaurant,Ramen Restaurant
22,Little Italy,Sushi Restaurant,Japanese Restaurant,American Restaurant,Korean Restaurant,Steakhouse,Smoothie Shop,Seafood Restaurant,Sake Bar,Restaurant,Ramen Restaurant
23,Soho,Sushi Restaurant,Japanese Restaurant,American Restaurant,Korean Restaurant,Steakhouse,Smoothie Shop,Seafood Restaurant,Sake Bar,Restaurant,Ramen Restaurant
24,West Village,Sushi Restaurant,Japanese Restaurant,Vegetarian / Vegan Restaurant,Fish Market,Sake Bar,Asian Restaurant,Noodle House,Steakhouse,Smoothie Shop,Seafood Restaurant
27,Gramercy,Sushi Restaurant,Japanese Restaurant,Vegetarian / Vegan Restaurant,Chinese Restaurant,Deli / Bodega,Noodle House,Steakhouse,Smoothie Shop,Seafood Restaurant,Sake Bar
28,Battery Park City,Sushi Restaurant,Japanese Restaurant,Noodle House,American Restaurant,Korean Restaurant,Steakhouse,Smoothie Shop,Seafood Restaurant,Sake Bar,Restaurant
31,Noho,Sushi Restaurant,Japanese Restaurant,Fish Market,American Restaurant,Korean Restaurant,Steakhouse,Smoothie Shop,Seafood Restaurant,Sake Bar,Restaurant


In [27]:
manhattan_merged.loc[manhattan_merged['Cluster Labels'] == 1, manhattan_merged.columns[[1] + list(range(5, manhattan_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
8,Upper East Side,Sushi Restaurant,Japanese Restaurant,Asian Restaurant,Vegetarian / Vegan Restaurant,Fish Market,Noodle House,Steakhouse,Smoothie Shop,Seafood Restaurant,Sake Bar
10,Lenox Hill,Sushi Restaurant,Asian Restaurant,Japanese Restaurant,Vegetarian / Vegan Restaurant,Fish Market,Noodle House,Steakhouse,Smoothie Shop,Seafood Restaurant,Sake Bar
14,Clinton,Sushi Restaurant,Japanese Restaurant,Chinese Restaurant,Cocktail Bar,Seafood Restaurant,Asian Restaurant,Korean Restaurant,American Restaurant,Ramen Restaurant,Steakhouse
15,Midtown,Sushi Restaurant,Japanese Restaurant,American Restaurant,Seafood Restaurant,Restaurant,Ramen Restaurant,Asian Restaurant,Vegetarian / Vegan Restaurant,Cocktail Bar,Bakery
16,Murray Hill,Sushi Restaurant,Japanese Restaurant,Vegetarian / Vegan Restaurant,Bakery,Chinese Restaurant,Restaurant,Asian Restaurant,Ramen Restaurant,Noodle House,Steakhouse
17,Chelsea,Sushi Restaurant,Japanese Restaurant,Asian Restaurant,Vegetarian / Vegan Restaurant,Fish Market,Noodle House,Steakhouse,Smoothie Shop,Seafood Restaurant,Sake Bar
34,Sutton Place,Sushi Restaurant,Japanese Restaurant,Asian Restaurant,Steakhouse,Deli / Bodega,Seafood Restaurant,American Restaurant,Noodle House,Smoothie Shop,Sake Bar
36,Tudor City,Sushi Restaurant,Asian Restaurant,Karaoke Bar,Japanese Restaurant,Steakhouse,Vegetarian / Vegan Restaurant,Fish Market,Grocery Store,Hawaiian Restaurant,Indian Chinese Restaurant
39,Hudson Yards,Sushi Restaurant,Japanese Restaurant,Cocktail Bar,Asian Restaurant,Korean Restaurant,American Restaurant,Noodle House,Steakhouse,Smoothie Shop,Seafood Restaurant


In [28]:
manhattan_merged.loc[manhattan_merged['Cluster Labels'] == 2, manhattan_merged.columns[[1] + list(range(5, manhattan_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Marble Hill,Sushi Restaurant,American Restaurant,Asian Restaurant,Steakhouse,Smoothie Shop,Seafood Restaurant,Sake Bar,Restaurant,Ramen Restaurant,Noodle House
3,Inwood,Sushi Restaurant,American Restaurant,Asian Restaurant,Steakhouse,Smoothie Shop,Seafood Restaurant,Sake Bar,Restaurant,Ramen Restaurant,Noodle House
4,Hamilton Heights,Sushi Restaurant,American Restaurant,Asian Restaurant,Steakhouse,Smoothie Shop,Seafood Restaurant,Sake Bar,Restaurant,Ramen Restaurant,Noodle House
6,Central Harlem,Sushi Restaurant,American Restaurant,Asian Restaurant,Steakhouse,Smoothie Shop,Seafood Restaurant,Sake Bar,Restaurant,Ramen Restaurant,Noodle House
7,East Harlem,Sushi Restaurant,American Restaurant,Asian Restaurant,Steakhouse,Smoothie Shop,Seafood Restaurant,Sake Bar,Restaurant,Ramen Restaurant,Noodle House
11,Roosevelt Island,Sushi Restaurant,Asian Restaurant,American Restaurant,Steakhouse,Smoothie Shop,Seafood Restaurant,Sake Bar,Restaurant,Ramen Restaurant,Noodle House
21,Tribeca,Sushi Restaurant,Noodle House,American Restaurant,Asian Restaurant,Steakhouse,Smoothie Shop,Seafood Restaurant,Sake Bar,Restaurant,Ramen Restaurant
37,Stuyvesant Town,Sushi Restaurant,American Restaurant,Asian Restaurant,Steakhouse,Smoothie Shop,Seafood Restaurant,Sake Bar,Restaurant,Ramen Restaurant,Noodle House


In [29]:
manhattan_merged.loc[manhattan_merged['Cluster Labels'] == 3, manhattan_merged.columns[[1] + list(range(5, manhattan_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,Washington Heights,Sushi Restaurant,Japanese Restaurant,American Restaurant,Korean Restaurant,Steakhouse,Smoothie Shop,Seafood Restaurant,Sake Bar,Restaurant,Ramen Restaurant
12,Upper West Side,Sushi Restaurant,Japanese Restaurant,Grocery Store,Asian Restaurant,American Restaurant,Noodle House,Steakhouse,Smoothie Shop,Seafood Restaurant,Sake Bar


In [30]:
manhattan_merged.loc[manhattan_merged['Cluster Labels'] == 4, manhattan_merged.columns[[1] + list(range(5, manhattan_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
5,Manhattanville,Sushi Restaurant,Japanese Restaurant,American Restaurant,Korean Restaurant,Steakhouse,Smoothie Shop,Seafood Restaurant,Sake Bar,Restaurant,Ramen Restaurant
9,Yorkville,Sushi Restaurant,Japanese Restaurant,Vegetarian / Vegan Restaurant,Fish Market,Indian Chinese Restaurant,Asian Restaurant,Noodle House,Steakhouse,Smoothie Shop,Seafood Restaurant
13,Lincoln Square,Sushi Restaurant,Japanese Restaurant,Chinese Restaurant,Smoothie Shop,Grocery Store,American Restaurant,Noodle House,Steakhouse,Seafood Restaurant,Sake Bar
25,Manhattan Valley,Sushi Restaurant,Hawaiian Restaurant,Japanese Restaurant,American Restaurant,Korean Restaurant,Steakhouse,Smoothie Shop,Seafood Restaurant,Sake Bar,Restaurant
26,Morningside Heights,Sushi Restaurant,Hawaiian Restaurant,Japanese Restaurant,American Restaurant,Korean Restaurant,Steakhouse,Smoothie Shop,Seafood Restaurant,Sake Bar,Restaurant
29,Financial District,Sushi Restaurant,Japanese Restaurant,American Restaurant,Korean Restaurant,Steakhouse,Smoothie Shop,Seafood Restaurant,Sake Bar,Restaurant,Ramen Restaurant
30,Carnegie Hill,Sushi Restaurant,Japanese Restaurant,Vegetarian / Vegan Restaurant,Indian Chinese Restaurant,Asian Restaurant,Noodle House,Steakhouse,Smoothie Shop,Seafood Restaurant,Sake Bar
33,Midtown South,Sushi Restaurant,Japanese Restaurant,Bakery,Chinese Restaurant,Restaurant,Asian Restaurant,Ramen Restaurant,American Restaurant,Noodle House,Steakhouse
38,Flatiron,Sushi Restaurant,Japanese Restaurant,Vegetarian / Vegan Restaurant,Chinese Restaurant,Deli / Bodega,Asian Restaurant,Noodle House,Steakhouse,Smoothie Shop,Seafood Restaurant
